<a href="https://colab.research.google.com/github/oorange-ocean/thermx-data/blob/main/%E7%81%AB%E7%94%B5%E6%95%B0%E6%8D%AE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初始化

In [ ]:
# @title 导入必要的库和设置环境
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

# 挂载 Google Drive
drive.mount('/content/drive')

# 定义文件路径
DATA_PATH = '/content/drive/My Drive/data_converted.tsv'
SAVE_PATH = '/content/drive/My Drive/cleaned_results.csv'

In [ ]:
# @title 加载数据
df = pd.read_csv(DATA_PATH, sep='\t')
print("数据预览：", df.head())
print("\n数据信息：", df.info())

In [ ]:
#@title 安装中文字体
!wget -O simhei.ttf "https://github.com/StellarCN/scp_zh/raw/master/fonts/SimHei.ttf"

In [ ]:
!file simhei.ttf

## 异常值检测与清理

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import skew
from scipy.signal import find_peaks

matplotlib.font_manager.fontManager.addfont('simhei.ttf')
matplotlib.rc('font', family='SimHei')

def analyze_distribution(df, column_name, bins=50, low_value_threshold=5.0):
    """分析数据分布并推荐异常值检测方法"""
    data = df[column_name].dropna()
    if len(data) == 0:
        print(f"列 '{column_name}' 无有效数据（全为 NaN），跳过分析")
        return None

    hist, bin_edges = np.histogram(data, bins=bins, density=True)
    peaks, _ = find_peaks(hist, prominence=0.1)
    data_skew = skew(data)
    low_value_ratio = (data < low_value_threshold).sum() / len(data) * 100

    print(f"列 '{column_name}' 分布分析：")
    print(f"- 有效数据量：{len(data)}")
    print(f"- 最小值：{data.min():.2f}, 最大值：{data.max():.2f}")
    print(f"- 低值区 (<{low_value_threshold}) 比例：{low_value_ratio:.2f}%")
    print(f"- 偏度：{data_skew:.2f}, 峰值数量：{len(peaks)}")

    if len(peaks) > 1:
        method = 'multi_peak'
        print("推荐方法：多峰处理（分段检测）")
    elif low_value_ratio > 40 and abs(data_skew) > 1:
        method = 'low_value_dense'
        print("推荐方法：低值密集区处理（分段检测）")
    elif abs(data_skew) > 1:
        method = 'percentile'
        print("推荐方法：百分位法（偏态分布）")
    else:
        method = 'IQR'
        print("推荐方法：IQR（均匀或近似正态分布）")

    return method

def detect_and_clean_outliers_auto(df, column_name, low_value_threshold=5.0, iqr_multiplier=1.5, percentile_bounds=(1, 99), high_outlier_threshold=5.0, max_outlier_ratio=5.0):
    """根据分布自动选择方法检测和清理异常值，详略得当"""
    print(f"\n检查列 '{column_name}' 的数据状态：")
    print(f"- 总数据量：{len(df[column_name])}, NaN 数量：{df[column_name].isna().sum()}")

    data = df[column_name].dropna()
    if len(data) == 0:
        print(f"列 '{column_name}' 无有效数据（全为 NaN），跳过处理")
        return False, None, None, None

    method = analyze_distribution(df, column_name, low_value_threshold=low_value_threshold)
    if method is None:
        return False, None, None, None

    outliers = pd.Series(dtype=float)
    low_data = None
    high_data = None
    split_point = None

    if method == 'multi_peak':
        # 计算直方图
        hist, bin_edges = np.histogram(data, bins=50, density=True)
        peaks, _ = find_peaks(hist, prominence=0.1)

        if len(peaks) > 1:
            # 改进：找到峰值之间的谷值作为分段点
            valleys, _ = find_peaks(-hist, prominence=0.05)  # 对直方图取反，找谷值
            if len(valleys) > 0:
                # 选择最深的谷值（通常在中间的谷值）
                valley_idx = valleys[np.argmin(hist[valleys])]
                split_point = bin_edges[valley_idx]
            else:
                # 如果找不到谷值，退回到原方法
                split_point = (bin_edges[peaks[0]] + bin_edges[peaks[-1]]) / 2

            low_data = data[data <= split_point]
            high_data = data[data > split_point]
            outliers_idx = []

            # 动态调整 iqr_multiplier
            for segment, label in [(low_data, '低值段'), (high_data, '高值段')]:
                if len(segment) > 0:
                    segment_skew = skew(segment)
                    # 如果段内偏度较大，增大 iqr_multiplier
                    adjusted_iqr_multiplier = iqr_multiplier + 0.5 * abs(segment_skew) if abs(segment_skew) > 0.5 else iqr_multiplier
                    print(f"- {label}：调整后的 IQR 倍数={adjusted_iqr_multiplier:.2f}")

                    Q1 = segment.quantile(0.25)
                    Q3 = segment.quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - adjusted_iqr_multiplier * IQR
                    upper_bound = Q3 + adjusted_iqr_multiplier * IQR
                    segment_outliers = segment[(segment < lower_bound) | (segment > upper_bound)]
                    outliers_idx.extend(segment_outliers.index)

            outliers = data.loc[outliers_idx]
            outlier_ratio = len(outliers) / len(df) * 100

            # 如果异常值比例过高，逐步放宽条件
            while outlier_ratio > max_outlier_ratio and adjusted_iqr_multiplier < 3.0:
                print(f"异常值占比 {outlier_ratio:.2f}% 过高，尝试放宽 IQR 倍数...")
                adjusted_iqr_multiplier += 0.5
                outliers_idx = []
                for segment, label in [(low_data, '低值段'), (high_data, '高值段')]:
                    if len(segment) > 0:
                        Q1 = segment.quantile(0.25)
                        Q3 = segment.quantile(0.75)
                        IQR = Q3 - Q1
                        lower_bound = Q1 - adjusted_iqr_multiplier * IQR
                        upper_bound = Q3 + adjusted_iqr_multiplier * IQR
                        segment_outliers = segment[(segment < lower_bound) | (segment > upper_bound)]
                        outliers_idx.extend(segment_outliers.index)
                outliers = data.loc[outliers_idx]
                outlier_ratio = len(outliers) / len(df) * 100

            df.loc[outliers_idx, column_name] = np.nan

    elif method == 'low_value_dense':
        low_mask = df[column_name] < low_value_threshold
        high_data = data[~low_mask.reindex(data.index, fill_value=False)]
        if len(high_data) > 0:
            Q1 = high_data.quantile(0.25)
            Q3 = high_data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - iqr_multiplier * IQR
            upper_bound = Q3 + iqr_multiplier * IQR
            outliers = high_data[(high_data < lower_bound) | (high_data > upper_bound)]
            df.loc[outliers.index, column_name] = np.nan

    elif method == 'percentile':
        lower_bound, upper_bound = np.percentile(data, percentile_bounds)
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        df.loc[outliers.index, column_name] = np.nan

    else:  # IQR
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR
        outliers = data[(data < lower_bound) | (data > upper_bound)]
        df.loc[outliers.index, column_name] = np.nan

    outlier_count = len(outliers)
    outlier_ratio = outlier_count / len(df) * 100

    if outlier_count == 0:
        print(f"列 '{column_name}' 无异常值，跳过详细日志和绘图")
        return False, None, None, None

    print(f"检测到 {outlier_count} 个异常值，占比 {outlier_ratio:.2f}%")

    if outlier_ratio > high_outlier_threshold:
        print(f"警告：异常值占比超过 {high_outlier_threshold}%，以下为详细信息：")
        if method == 'multi_peak':
            print(f"- 分段点：{split_point:.2f}")
            for segment, label in [(low_data, '低值段'), (high_data, '高值段')]:
                if len(segment) > 0:
                    print(f"- {label}：数据量={len(segment)}, 最小值={segment.min():.2f}, 最大值={segment.max():.2f}")
                    print(f"  Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}, 下限={lower_bound:.2f}, 上限={upper_bound:.2f}")
        elif method == 'low_value_dense':
            print(f"- 非低值部分：数据量={len(high_data)}, 最小值={high_data.min():.2f}, 最大值={high_data.max():.2f}")
            print(f"  Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}, 下限={lower_bound:.2f}, 上限={upper_bound:.2f}")
        elif method == 'percentile':
            print(f"- 百分位范围：下限={lower_bound:.2f}, 上限={upper_bound:.2f}")
        else:
            print(f"- IQR 参数：Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}, 下限={lower_bound:.2f}, 上限={upper_bound:.2f}")

    print(f"\n列 '{column_name}' 的异常值样本：")
    sample_size = min(5, len(outliers) // 2)
    print("最小值端：", outliers.nsmallest(sample_size).to_list())
    print("最大值端：", outliers.nlargest(sample_size).to_list())
    return True, method, low_data, high_data

def plot_distribution(df, column_name, method, low_data=None, high_data=None):
    """绘制箱线图和直方图，支持分段展示"""
    data = df[column_name].dropna()
    if len(data) == 0:
        print(f"列 '{column_name}' 无有效数据（全为 NaN），跳过绘图")
        return

    if method == 'multi_peak' and low_data is not None and high_data is not None:
        plt.figure(figsize=(12, 10))

        plt.subplot(2, 2, 1)
        plt.boxplot(low_data)
        plt.title(f"Boxplot of {column_name} (Low Segment)")

        plt.subplot(2, 2, 2)
        plt.hist(low_data, bins=50, color='blue', alpha=0.7)
        plt.title(f"Histogram of {column_name} (Low Segment)")

        plt.subplot(2, 2, 3)
        plt.boxplot(high_data)
        plt.title(f"Boxplot of {column_name} (High Segment)")

        plt.subplot(2, 2, 4)
        plt.hist(high_data, bins=50, color='green', alpha=0.7)
        plt.title(f"Histogram of {column_name} (High Segment)")

        plt.tight_layout()
        plt.show()
    else:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.boxplot(data)
        plt.title(f"Boxplot of {column_name}")

        plt.subplot(1, 2, 2)
        plt.hist(data, bins=50)
        plt.title(f"Histogram of {column_name}")
        plt.show()

# 处理所有数值列
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    print(f"\n=== 处理列 '{col}' ===")
    has_outliers, method, low_data, high_data = detect_and_clean_outliers_auto(
        df,
        col,
        low_value_threshold=5.0,
        iqr_multiplier=1.5,
        percentile_bounds=(1, 99),
        high_outlier_threshold=5.0,
        max_outlier_ratio=3.0       # 新增参数：目标异常值比例上限
    )
    if has_outliers:
        plot_distribution(df, col, method, low_data, high_data)

In [ ]:
# @title 保存异常清理数据
df.to_csv(SAVE_PATH, index=False)
print(f"处理后的数据已保存至：{SAVE_PATH}")
print("\n最终数据预览：", df.head())

In [ ]:
# @title 稳态运行区间提取
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter  # 用于数据平滑

# 确保中文字体已加载（基于你之前的代码）
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 定义稳态检测函数
def detect_steady_state(df, column_name='机组负荷', window_size=30, std_threshold=5.0, min_duration=10):
    data = df[column_name].dropna()
    if len(data) == 0:
        print(f"列 '{column_name}' 无有效数据，跳过稳态检测")
        return []

    smoothed_data = savgol_filter(data, window_length=window_size, polyorder=2)
    rolling_std = data.rolling(window=window_size, center=True).std()
    is_steady = rolling_std < std_threshold

    steady_intervals = []
    start_idx = None

    for i in range(len(is_steady)):
        if is_steady.iloc[i] and start_idx is None:
            start_idx = i
        elif not is_steady.iloc[i] and start_idx is not None:
            end_idx = i - 1
            if (end_idx - start_idx + 1) >= min_duration:
                steady_intervals.append((start_idx, end_idx))
            start_idx = None

    if start_idx is not None and (len(data) - start_idx) >= min_duration:
        steady_intervals.append((start_idx, len(data) - 1))

    # 定量评估
    total_length = len(data)
    steady_length = sum(end - start + 1 for start, end in steady_intervals)
    coverage = steady_length / total_length * 100
    stability = [data.iloc[start:end+1].std() for start, end in steady_intervals]

    print(f"检测到 {len(steady_intervals)} 个稳态运行区间")
    print(f"稳态覆盖率: {coverage:.2f}%")
    print(f"各稳态区间标准差: {[f'{s:.2f}' for s in stability]}")
    for idx, (start, end) in enumerate(steady_intervals):
        start_time = df['时间'].iloc[start]
        end_time = df['时间'].iloc[end]
        avg_load = data.iloc[start:end+1].mean()
        print(f"区间 {idx+1}: {start_time} 至 {end_time}, 平均负荷: {avg_load:.2f}")

    return steady_intervals


# 绘制稳态区间图
def plot_steady_state(df, column_name='机组负荷', steady_intervals=None):
    """
    绘制目标列的时间序列图并标注稳态区间
    """
    data = df[column_name].dropna()
    time = pd.to_datetime(df['时间'])

    plt.figure(figsize=(15, 6))
    plt.plot(time, data, label=f'{column_name} (原始数据)', alpha=0.5)

    if steady_intervals:
        for start, end in steady_intervals:
            plt.axvspan(time.iloc[start], time.iloc[end], color='green', alpha=0.3, label='稳态区间' if start == steady_intervals[0][0] else "")

    plt.title(f'{column_name} 时间序列与稳态区间')
    plt.xlabel('时间')
    plt.ylabel(column_name)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 执行稳态检测
steady_intervals = detect_steady_state(
    df,
    column_name='机组负荷',      # 使用机组负荷作为稳态判断依据
    window_size=30,             # 滑动窗口大小（可根据数据频率调整）
    std_threshold=5.0,          # 标准差阈值（可根据实际情况调整）
    min_duration=10             # 最小稳态区间长度（数据点数）
)

# 绘制结果
plot_steady_state(df, '机组负荷', steady_intervals)


In [ ]:
# @title 定量评估稳态区间提取（基于固定阈值的对比方法）
def detect_steady_state_threshold(df, column_name='机组负荷', threshold=5.0, min_duration=10):
    data = df[column_name].dropna()
    diff = data.diff().abs()  # 计算相邻点差值
    is_steady = diff < threshold

    steady_intervals = []
    start_idx = None

    for i in range(len(is_steady)):
        if is_steady.iloc[i] and start_idx is None:
            start_idx = i
        elif not is_steady.iloc[i] and start_idx is not None:
            end_idx = i - 1
            if (end_idx - start_idx + 1) >= min_duration:
                steady_intervals.append((start_idx, end_idx))
            start_idx = None

    if start_idx is not None and (len(data) - start_idx) >= min_duration:
        steady_intervals.append((start_idx, len(data) - 1))

    print(f"[固定阈值法] 检测到 {len(steady_intervals)} 个稳态运行区间")
    return steady_intervals

# 对比实验
steady_intervals_threshold = detect_steady_state_threshold(df, '机组负荷', threshold=5.0, min_duration=10)
plot_steady_state(df, '机组负荷', steady_intervals_threshold)

In [ ]:
# @title 保存稳态区间数据
steady_df = pd.DataFrame()
for idx, (start, end) in enumerate(steady_intervals, 1):
    steady_segment = df.iloc[start:end+1].copy()
    steady_segment['稳态区间编号'] = idx  # 从 1 开始递增
    steady_df = pd.concat([steady_df, steady_segment])

STEADY_SAVE_PATH = '/content/drive/My Drive/steady_state_data.csv'
steady_df.to_csv(STEADY_SAVE_PATH, index=False)
print(f"稳态运行区间数据已保存至：{STEADY_SAVE_PATH}")
print("\n稳态数据预览：", steady_df.head())

In [ ]:
# @title 特征提取（通用版，使用 TS2Vec）
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import torch
import sys
from tqdm import tqdm
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import variation

# 挂载 Google Drive
drive.mount('/content/drive')

# 定义文件路径
STEADY_SAVE_PATH = '/content/drive/My Drive/steady_state_data.csv'
FEATURE_SAVE_PATH = '/content/drive/My Drive/ts2vec_features.csv'

# 安装依赖
!pip install torch numpy pandas scipy tqdm -q

# 下载并导入 TS2Vec
!git clone https://github.com/zhihanyue/ts2vec.git -q
sys.path.append('/content/ts2vec')
from ts2vec import TS2Vec

# 加载数据
print("加载稳态数据...")
steady_df = pd.read_csv(STEADY_SAVE_PATH)
print("稳态数据预览：", steady_df.head())

# 动态计算稳态区间的数量
num_steady_intervals = steady_df['稳态区间编号'].nunique()
print(f"\n动态计算的稳态区间数量：{num_steady_intervals}")

# 验证稳态区间编号
print("\n检查 '稳态区间编号' 的分布：")
print("唯一值数量：", steady_df['稳态区间编号'].nunique())
print("最小值：", steady_df['稳态区间编号'].min())
print("最大值：", steady_df['稳态区间编号'].max())
print("前 10 个唯一值：", sorted(steady_df['稳态区间编号'].unique())[:10])

# 如果编号不连续或最大值异常，重新生成连续编号
# 期望编号从 1 到 num_steady_intervals
if steady_df['稳态区间编号'].min() < 1 or steady_df['稳态区间编号'].max() != num_steady_intervals:
    print("检测到 '稳态区间编号' 异常，重新生成连续编号...")
    grouped = steady_df.groupby('稳态区间编号')
    steady_df['稳态区间编号'] = steady_df['稳态区间编号'].map(
        {old_id: new_id for new_id, (old_id, _) in enumerate(grouped, 1)}
    )
    print("修复后 '稳态区间编号' 的分布：")
    print("唯一值数量：", steady_df['稳态区间编号'].nunique())
    print("最小值：", steady_df['稳态区间编号'].min())
    print("最大值：", steady_df['稳态区间编号'].max())

In [ ]:
# @title 特征重要性分析与降维
numeric_cols = steady_df.select_dtypes(include=[np.number]).columns.drop('稳态区间编号')
print(f"共有 {len(numeric_cols)} 个数值列")

# 计算变异系数
def calculate_cv(df, cols):
    cv_dict = {}
    for col in tqdm(cols, desc="计算变异系数"):
        data = df[col].dropna()
        if len(data) > 0:
            cv_dict[col] = variation(data)
    return cv_dict

cv_dict = calculate_cv(steady_df, numeric_cols)
cv_df = pd.DataFrame(list(cv_dict.items()), columns=['特征', '变异系数']).sort_values('变异系数', ascending=False)

# 计算互信息（以‘机组负荷’为目标）
def calculate_mutual_info(df, target_col, feature_cols):
    X = df[feature_cols].fillna(0)
    y = df[target_col].fillna(0)
    mi_scores = mutual_info_regression(X, y)
    return pd.DataFrame({'特征': feature_cols, '互信息': mi_scores}).sort_values('互信息', ascending=False)

target_col = '机组负荷'
mi_df = calculate_mutual_info(steady_df, target_col, numeric_cols.drop(target_col))

# 计算灰色关联度
def calculate_grey_relation(df, target_col, feature_cols):
    X = df[feature_cols].fillna(0)
    y = df[target_col].fillna(0)
    X_norm = (X - X.min()) / (X.max() - X.min())
    y_norm = (y - y.min()) / (y.max() - y.min())
    diff = np.abs(X_norm.values - y_norm.values[:, None])
    rho = 0.5
    min_diff = diff.min()
    max_diff = diff.max()
    grey_coeff = (min_diff + rho * max_diff) / (diff + rho * max_diff)
    grey_relation = grey_coeff.mean(axis=0)
    return pd.DataFrame({'特征': feature_cols, '灰色关联度': grey_relation}).sort_values('灰色关联度', ascending=False)

grey_df = calculate_grey_relation(steady_df, target_col, numeric_cols.drop(target_col))

# 必须包含的重点变量
must_include = ['主汽压力', '再热汽压力']  # 如果还有其他变量，可以加在这里

# 选择关键特征（Top 8，但确保包含 must_include）
top_cv = cv_df['特征'].head(4).tolist()
top_mi = mi_df['特征'].head(4).tolist()
top_grey = grey_df['特征'].head(4).tolist()

# 合并特征并确保 must_include 被包含
feature_cols = list(set(top_cv + top_mi + top_grey + must_include))
print(f"选择的 {len(feature_cols)} 个关键特征（含必须变量 {must_include}）：", feature_cols)

# @title 数据准备（降采样）
def prepare_ts2vec_data(df, feature_cols, group_col='稳态区间编号', max_len=500):
    grouped = df.groupby(group_col)
    data_list = []

    for _, group in tqdm(grouped, desc="准备 TS2Vec 数据"):
        features = group[feature_cols].values
        if len(features) > max_len:
            indices = np.linspace(0, len(features) - 1, max_len).astype(int)
            features = features[indices]
        data_list.append(features)

    padded_data = np.zeros((len(data_list), max_len, len(feature_cols)))
    for i, seq in tqdm(enumerate(data_list), total=len(data_list), desc="填充数据"):
        padded_data[i, :len(seq), :] = seq

    return padded_data

ts2vec_data = prepare_ts2vec_data(steady_df, feature_cols, max_len=500)
print("TS2Vec 输入数据形状：", ts2vec_data.shape)

In [ ]:

# @title 训练 TS2Vec 模型
model = TS2Vec(
    input_dims=len(feature_cols),
    device='cpu',
    output_dims=64,
    hidden_dims=32,
    depth=5,
    batch_size=8,
    lr=0.001
)

print("开始训练 TS2Vec 模型...")
loss_log = model.fit(
    ts2vec_data,
    n_epochs=50,
    verbose=True
)

# @title 特征提取
print("提取特征...")
features = model.encode(ts2vec_data, encoding_window='full_series')
print("提取的特征形状：", features.shape)

# 构建特征数据框
feature_df = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(features.shape[1])])
# 手动生成连续编号
feature_df['稳态区间编号'] = np.arange(1, len(feature_df) + 1)

# 可视化
plt.figure(figsize=(10, 6))
plt.scatter(feature_df['feature_0'], feature_df['feature_1'], c=feature_df['稳态区间编号'], cmap='viridis')
plt.title('TS2Vec 提取的特征可视化（前两个维度）')
plt.xlabel('Feature 0')
plt.ylabel('Feature 1')
plt.colorbar(label='稳态区间编号')
plt.show()

# 保存结果
print("保存特征结果...")
feature_df.to_csv(FEATURE_SAVE_PATH, index=False)
print(f"特征提取结果已保存至：{FEATURE_SAVE_PATH}")
print("\n特征数据预览：", feature_df.head())

## 聚类

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from google.colab import drive

# 挂载 Google Drive
drive.mount('/content/drive')

# 文件路径
STEADY_SAVE_PATH = '/content/drive/My Drive/steady_state_data.csv'
TS2VEC_FEATURE_PATH = '/content/drive/My Drive/ts2vec_features.csv'
CLUSTER_SAVE_PATH = '/content/drive/My Drive/clustering_data_ts2vec.csv'

# 加载数据
print("加载稳态数据和 TS2Vec 特征...")
steady_df = pd.read_csv(STEADY_SAVE_PATH)
feature_df = pd.read_csv(TS2VEC_FEATURE_PATH)
print("稳态数据预览：", steady_df.head())
print("TS2Vec 特征预览：", feature_df.head())

# 计算原始数据的平均特征
print("计算原始数据的平均特征...")
steady_grouped = steady_df.groupby('稳态区间编号').agg({
    '机组负荷': 'mean',
    '汽轮机热耗率q': 'mean'
}).reset_index()
steady_grouped.rename(columns={
    '机组负荷': '平均机组负荷',
    '汽轮机热耗率q': '平均热耗率'
}, inplace=True)
print("平均特征预览：", steady_grouped.head())

# 准备 TS2Vec 特征
print("准备 TS2Vec 特征用于聚类...")
X = feature_df.drop(columns=['稳态区间编号']).values
print("特征向量形状：", X.shape)

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA 降维可视化检查特征区分度
print("检查 TS2Vec 特征的区分度（PCA 降维）...")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=50)
plt.title('TS2Vec 特征 PCA 降维可视化')
plt.xlabel('主成分 1')
plt.ylabel('主成分 2')
plt.grid(True)
plt.show()
print(f"前两个主成分解释的方差比例：{pca.explained_variance_ratio_.sum():.3f}")

# 方法 1：改进 K-Means 聚类
print("\n=== 方法 1：改进 K-Means 聚类 ===")
silhouette_scores = []
K = range(2, 10)  # 缩小范围，专注少量簇
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)
    print(f"簇数 {k} 的轮廓系数：{score:.3f}")

n_clusters = 5  # 手动指定为 5，基于业务需求而非仅轮廓系数
print(f"手动选择簇数：{n_clusters}")
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
feature_df['Cluster_KMeans'] = kmeans.fit_predict(X_scaled)

# 方法 2：尝试 DBSCAN 聚类
print("\n=== 方法 2：DBSCAN 聚类 ===")
dbscan = DBSCAN(eps=2.0, min_samples=5)  # eps 和 min_samples 可调
feature_df['Cluster_DBSCAN'] = dbscan.fit_predict(X_scaled)
n_clusters_dbscan = len(set(feature_df['Cluster_DBSCAN'])) - (1 if -1 in feature_df['Cluster_DBSCAN'] else 0)
print(f"DBSCAN 检测到的簇数：{n_clusters_dbscan}")
print(f"噪声点数量（标签 -1）：{(feature_df['Cluster_DBSCAN'] == -1).sum()}")

# 合并 K-Means 结果到平均特征数据
clustered_df = steady_grouped.merge(feature_df[['稳态区间编号', 'Cluster_KMeans', 'Cluster_DBSCAN']],
                                    on='稳态区间编号',
                                    how='left')
clustered_df = clustered_df.dropna(subset=['Cluster_KMeans'])  # 移除未匹配的样本

# 生成语义标签（基于 K-Means）
def assign_semantic_labels(df, cluster_col='Cluster_KMeans'):
    cluster_stats = df.groupby(cluster_col).agg({
        '平均机组负荷': 'mean',
        '平均热耗率': 'mean'
    })
    # 动态调整阈值基于数据范围
    load_mean, load_std = steady_grouped['平均机组负荷'].mean(), steady_grouped['平均机组负荷'].std()
    heat_mean, heat_std = steady_grouped['平均热耗率'].mean(), steady_grouped['平均热耗率'].std()
    labels = []
    for idx, row in cluster_stats.iterrows():
        load = row['平均机组负荷']
        heat = row['平均热耗率']
        load_label = ("高负荷" if load > load_mean + load_std else
                      "低负荷" if load < load_mean - load_std else "中负荷")
        heat_label = ("高效" if heat < heat_mean - heat_std else
                      "低效" if heat > heat_mean + heat_std else "中等效率")
        labels.append(f"{load_label}-{heat_label}")
    df['Semantic_Label'] = df[cluster_col].map(dict(zip(cluster_stats.index, labels)))
    return df

clustered_df = assign_semantic_labels(clustered_df, 'Cluster_KMeans')
print("K-Means 带语义标签的结果：",
      clustered_df[['Cluster_KMeans', 'Semantic_Label', '平均机组负荷', '平均热耗率']].drop_duplicates())

# 可视化 K-Means 结果
plt.figure(figsize=(10, 6))
for cluster_id in clustered_df['Cluster_KMeans'].unique():
    cluster_data = clustered_df[clustered_df['Cluster_KMeans'] == cluster_id]
    plt.scatter(cluster_data['平均机组负荷'], cluster_data['平均热耗率'],
                label=cluster_data['Semantic_Label'].iloc[0], s=100)
plt.title(f'K-Means 聚类结果 (基于 TS2Vec 特征, 簇数: {n_clusters})')
plt.xlabel('平均机组负荷')
plt.ylabel('平均热耗率')
plt.legend(title='语义标签')
plt.grid(True)
plt.show()

# 可视化 DBSCAN 结果
plt.figure(figsize=(10, 6))
plt.scatter(clustered_df['平均机组负荷'], clustered_df['平均热耗率'],
            c=clustered_df['Cluster_DBSCAN'], cmap='tab10', s=100)
plt.title('DBSCAN 聚类结果 (基于 TS2Vec 特征)')
plt.xlabel('平均机组负荷')
plt.ylabel('平均热耗率')
plt.colorbar(label='聚类标签 (-1 为噪声)')
plt.grid(True)
plt.show()

# 保存结果
clustered_df.to_csv(CLUSTER_SAVE_PATH, index=False)
print(f"聚类结果已保存至：{CLUSTER_SAVE_PATH}")

# 输出统计信息
print("\nK-Means 每个聚类的稳态区间数量：")
print(clustered_df['Cluster_KMeans'].value_counts())
print("\nK-Means 每个聚类的特征统计：")
print(clustered_df.groupby('Cluster_KMeans').agg({
    '平均机组负荷': ['mean', 'std', 'min', 'max'],
    '平均热耗率': ['mean', 'std', 'min', 'max']
}))
print("\nDBSCAN 每个聚类的稳态区间数量：")
print(clustered_df['Cluster_DBSCAN'].value_counts())